In [71]:
import pandas as pd
import glob
import time
import numpy as np

import sys
from tweetple import TweetPle
sys.path.insert(0, '../../src/utils')
from funcs import *
from datetime import date, timedelta
from tqdm import tqdm

sys.path.insert(0, '../../src/utils')
from general import *
from scrape import *

In [2]:
"""Retrieve Content Shared By Influencers During Treatment

-- keywords: #FactsMatter, @AfricaCheck
-- start date: 13-03-23
-- end date:
"""

import pandas as pd
import glob
import time
import numpy as np

import sys
from tweetple import TweetPle
sys.path.insert(0, '../../src/utils')
from funcs import *
from datetime import date, timedelta
from tqdm import tqdm

sys.path.insert(0, '../../src/utils')
from general import *
from scrape import *


def start_date(days_ago):
    """Start date of scrape"""
    start = str(date.today() - timedelta(days_ago))
    start = start + "T00:00:00Z"

    return start


def end_date(days_ago):
    """Start date of scrape"""
    end = str(date.today() - timedelta(days_ago))
    end = end + "T23:59:59Z"

    return end


def scrape_tweets(accounts, path, bearer_token, days_ago):
    """Scrape Tweets"""
    end = end_date(days_ago - 6)
    TweetPle.TweetStreamer(
        accounts, bearer_token, path, start_date(days_ago), end
    ).main()

    return print("Content scraped")


def get_paths(days_ago, country):

    base = f"../../data/03-experiment/{country}/treatment/influencers/"
    rest = timedelta(days_ago - 6)
    datep = str(date.today() - rest)
    path_tw = base + "00-raw/twitter/" + datep

    return path_tw, base, datep


def tweet_types(path_read):
    df = read_files(path_read)
    df.referenced_tweets = df.referenced_tweets.fillna({i: {} for i in df.index})
    m = pd.DataFrame(df["referenced_tweets"].tolist())
    m = m[0].apply(pd.Series)
    m.replace(
        {
            "retweeted": "retweet",
            "replied_to": "reply",
            "quoted": "quote",
            np.nan: "tweet",
        },
        inplace=True,
    )
    df["type"] = m["type"]
    df = df.reset_index(drop=True)
    df = df.drop("referenced_tweets", axis=1)
    return df


def content_twitter(path):
    """Find Tweets containing #FactsMatter"""
    pattern = '|'.join([f'(?i){word}' for word in words_tweets])
    pattern_url = '|'.join([f'(?i){url}' for url in urls_list])
    # df = tweet_types(path)
    df = read_files(path)
    df = expand_column(df, "entities.hashtags")
    df.rename({"tag": "hashtag"}, axis=1, inplace=1)
    df = df.drop(["end", "start"], axis=1)
    df = expand_column(df, "entities.urls")
    df0 = df.loc[df["text"].str.contains("#factsmatter", case=False)]
    df0['campaign_hashtag'] = 1
    df1 = df.loc[df["text"].str.contains("@africacheck", case=False)]
    df1['campaign_hashtag'] = 1
    df2 = df.loc[df["text"].str.contains(pattern, case=False)]
    df2 = df2[~df2['id'].isin(list(list(df0.id)+list(df1.id)))]
    df2['campaign_hashtag'] = 0
    df3 = df.loc[df["expanded_url"].str.contains(pattern_url, case=False, na=False)]
    df3 = df3[~df3['id'].isin(list(list(df0.id)+list(df1.id)))]
    df3['campaign_hashtag'] = 0
    df = df0.append(df1).append(df2).append(df3).drop_duplicates(subset="id").reset_index(drop=True)
    try:
        df["retweet"] = np.where(df.url.isna(), 1, 0)
    except:
        df["url"] = np.nan
        df["retweet"] = 1
    df = df[df["retweet"] != 1]
    df = df.drop("retweet", axis=1)
    df["tweet_url"] = "https://twitter.com/" + df.handle + "/status/" + df.id
    df.reset_index(drop=True,inplace=True)
    return df

def content_twitter_old(path):
    """Find Tweets containing #FactsMatter"""
    # df = tweet_types(path)
    df = read_files(path)
    df = expand_column(df, "entities.hashtags")
    df.rename({"tag": "hashtag"}, axis=1, inplace=1)
    df = df.drop(["end", "start"], axis=1)
    df0 = df.loc[df["text"].str.contains("#factsmatter", case=False)]
    df1 = df.loc[df["text"].str.contains("@africacheck", case=False)]
    df = df0.append(df1).drop_duplicates(subset="id").reset_index(drop=True)
    df = expand_column(df, "entities.urls")
    try:
        df["retweet"] = np.where(df.url.isna(), 1, 0)
    except:
        df["url"] = np.nan
        df["retweet"] = 1
    df = df[df["retweet"] != 1]
    df = df.drop("retweet", axis=1)
    df["tweet_url"] = "https://twitter.com/" + df.handle + "/status/" + df.id

    return df


def incremental_range(start, stop, step, inc):
    value = start
    while value < stop:
        yield value
        value += step
        step += inc


def create_list(df, start, step, inc):
    stop = len(df.columns)
    a_list = list(incremental_range(start, stop, step, inc))

    return a_list


def create_dictionaries(contents, n_contents, a_list, name1, name2):
    a_dict = dict(zip(contents, a_list))
    b_dict = dict(zip(n_contents, a_list))
    a_dict = {k: name1 + v for k, v in a_dict.items()}
    b_dict = {k: name2 + v for k, v in b_dict.items()}

    return a_dict, b_dict


def rename_columns(df, a_dict, b_dict):
    for key, value in a_dict.items():
        df.columns.values[key] = value
    for key, value in b_dict.items():
        df.columns.values[key] = value

    return df


def drop_duplicated_columns(df):
    df = df.loc[:, ~df.columns.duplicated()]

    return df


def problematic(df, t_col, n_col, str_find):
    prob = [col for col in df.columns if str_find in col]
    df["auxiliar"] = df[prob].sum(axis=1)
    df[n_col] = np.where((df["auxiliar"] == 0) & (df[t_col] == 1), 1, 0)
    df = df.drop(["auxiliar"], axis=1)

    return df




In [3]:
a = pd.read_parquet("/Users/joaquinbarrutia/Dropbox/Bolivia_Project/social-media-influencers-af/data/03-experiment/KE/treatment/influencers/00-raw/twitter/2023-05-07/2314837515.parquet")

In [23]:
pattern = '|'.join([f'(?i){word}' for word in words_tweets])
pattern_url = '|'.join([f'(?i){url}' for url in urls_list])

In [116]:
df = read_files("/Users/joaquinbarrutia/Dropbox/Bolivia_Project/social-media-influencers-af/data/03-experiment/KE/treatment/influencers/00-raw/twitter/2023-05-07/")

In [117]:
df[df['author_id']=='849395300']

,author_id,id,conversation_id,created_at,text,retweet_count,reply_count,like_count,quote_count,impression_count
0,849395300,1654352219762827266,1654352219762827266,2023-05-05T05:08:00.000Z,Be wary of WhatsApp posts purporting Kenya's F...,0.0,0.0,1.0,0.0,74.0
1,849395300,1653989106362269696,1653989106362269696,2023-05-04T05:05:07.000Z,RT @WIPO: The next chapter for the fashion ind...,19.0,0.0,0.0,0.0,0.0
2,849395300,1653754353486897153,1653754353486897153,2023-05-03T13:32:17.000Z,RT @UN: A free press is essential for exposing...,323.0,0.0,0.0,0.0,0.0
3,849395300,1653754296578502722,1653754296578502722,2023-05-03T13:32:04.000Z,Happy Article 34 of The Constitution of Kenya ...,0.0,0.0,1.0,0.0,63.0
4,849395300,1653673874221834241,1653673874221834241,2023-05-03T08:12:30.000Z,RT @UN: Attacks on journalists are attacks on ...,505.0,0.0,0.0,0.0,0.0
5,849395300,1653673682391146496,1653673682391146496,2023-05-03T08:11:44.000Z,RT @WIPO: Can the sun☀️ help you with your hom...,16.0,0.0,0.0,0.0,0.0
6,849395300,1653672375110492160,1653672375110492160,2023-05-03T08:06:32.000Z,RT @moneyacademyKE: Big win for consumers toda...,240.0,0.0,0.0,0.0,0.0
7,849395300,1653672016698802181,1653672016698802181,2023-05-03T08:05:07.000Z,RT @salilsethi: Most businesses don't die beca...,4090.0,0.0,0.0,0.0,0.0
8,849395300,1653266817631264771,1653266817631264771,2023-05-02T05:15:00.000Z,IT’S A SCAM! Use these straightforward steps t...,0.0,0.0,1.0,0.0,70.0
9,849395300,1653087464280539150,1653087464280539150,2023-05-01T17:22:19.000Z,RT @TheKSTS: The KSTS wishes all hardworking i...,2.0,0.0,0.0,0.0,0.0


In [115]:
df.groupby(['type']).count().tail(60)

,author_id,id,conversation_id,created_at,text,retweet_count,reply_count,like_count,quote_count,impression_count
type,,,,,,,,,,
quoted,712,712,712,712,712,712,712,712,712,712
replied_to,2109,2109,2109,2109,2109,2109,2109,2109,2109,2109
retweeted,4133,4133,4133,4133,4133,4133,4133,4133,4133,4133


In [119]:
def content_twitter(path):
    """Find Tweets containing #FactsMatter"""
    pattern = '|'.join([f'(?i){word}' for word in words_tweets])
    pattern_url = '|'.join([f'(?i){url}' for url in urls_list])
    # df = tweet_types(path)
    df = read_files(path)
    df = df[~df['text'].isnull()]
    #df = expand_column(df, "entities.hashtags")
    #df.rename({"tag": "hashtag"}, axis=1, inplace=1)
    #df = df.drop(["end", "start"], axis=1)
    #df = expand_column(df, "entities.urls")
    df0 = df.loc[df["text"].str.contains("#factsmatter", case=False)]
    df0['campaign_hashtag'] = 1
    df1 = df.loc[df["text"].str.contains("@africacheck", case=False)]
    df1['campaign_hashtag'] = 1
    df2 = df.loc[df["text"].str.contains(pattern, case=False)]
    df2 = df2[~df2['id'].isin(list(list(df0.id)+list(df1.id)))]
    df2['campaign_hashtag'] = 0
    #df3 = df.loc[df["expanded_url"].str.contains(pattern_url, case=False, na=False)]
    #df3 = df3[~df3['id'].isin(list(list(df0.id)+list(df1.id)))]
    #df3['campaign_hashtag'] = 0
    df = df0.append(df1).append(df2).drop_duplicates(subset="id").reset_index(drop=True)
    #df = df[df['type']!='retweeted']
    #try:
     #   df["retweet"] = np.where(df.url.isna(), 1, 0)
    #except:
     #   df["url"] = np.nan
      #  df["retweet"] = 1
    #df = df[df["retweet"] != 1]
    #df = df.drop("retweet", axis=1)
    #df["tweet_url"] = "https://twitter.com/" + df.handle + "/status/" + df.id
    df.reset_index(drop=True,inplace=True)
    return df


In [77]:
def content_twitter(path):
    """Find Tweets containing #FactsMatter"""
    pattern = '|'.join([f'(?i){word}' for word in words_tweets])
    pattern_url = '|'.join([f'(?i){url}' for url in urls_list])
    # df = tweet_types(path)
    df = read_files(path)
    df = df[~df['text'].isnull()]
    #df = expand_column(df, "entities.hashtags")
    #df.rename({"tag": "hashtag"}, axis=1, inplace=1)
    #df = df.drop(["end", "start"], axis=1)
    #df = expand_column(df, "entities.urls")
    df0 = df.loc[df["text"].str.contains("#factsmatter", case=False)]
    return df0

In [122]:
df_tw = content_twitter(f"/Users/joaquinbarrutia/Dropbox/Bolivia_Project/social-media-influencers-af/data/03-experiment/SA/treatment/influencers/00-raw/twitter/2023-05-07/")

<ipython-input-119-64fac4f38c2b>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df0['campaign_hashtag'] = 1
<ipython-input-119-64fac4f38c2b>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['campaign_hashtag'] = 1
<ipython-input-119-64fac4f38c2b>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df0.append(df1).append(df2).drop_duplicates(subset="id").reset_index(drop=True)
<ipython-input-119-64fac4f38c2b>:

In [83]:
import tweepy

In [84]:
Client = tweepy.Client(bearer_token=bearer)

In [ ]:
df = Client.search_all_tweets(query = 'from:1653300807779975171', start_tweets = start, end_tweets = end, max_results=500)

In [88]:
from time import sleep
import tweepy
import os

client = tweepy.Client(bearer_token=bearer, wait_on_rate_limit=True)

query = 'from:1653300807779975171'

for response in tweepy.Paginator(client.search_all_tweets, query=query, start_time = start, end_time = end, max_results=500):
    sleep(1)
    print(response)
    r = response

Response(data=None, includes={}, errors=[], meta={'result_count': 0})


In [100]:
paginator = tweepy.Paginator(
    client.search_all_tweets,               # The method you want to use
    query="from:44196397",                            # Some argument for this method
    end_time=end,       # Some argument for this method   
    start_time=start,     # Some argument for this method
    max_results=500                        # How many tweets asked per request
)

try: 
    for page in paginator:      # Default to inf
        print(page)
        print(page.data)                       # The tweets are here
        print(page.meta)                       # The count etc. are here
        print(page.includes)  
except tweepy.RateLimitError as exc:
    print('Rate limit!')

{}


In [1]:
start =  '2023-05-01T00:00:00Z'
end = '2023-05-07T00:00:00Z'
bearer = "AAAAAAAAAAAAAAAAAAAAAFpgZAEAAAAAbJS59UWzipi32ixd7LHtXov9olo%3D7gxD8Afshgj4munMXHLU08jzRdTpsAh4RZqq7VBofq1wAvkx1T"

In [4]:
TweetPle.TweetStreamer('1126820633244065792', bearer, '/Users/joaquinbarrutia/',start, end ).main()

0it [00:02, ?it/s]


KeyError: 'data'

In [40]:
df.drop_duplicates('author_id')

,author_id,id,conversation_id,created_at,type,text,retweet_count,reply_count,like_count,quote_count,impression_count
0,503359879,1655129258350575616,1655127830311899136,2023-05-07T08:35:40.000Z,replied_to,@TheMiddleBorn @wwwasike,0.0,1.0,22.0,0.0,45.0
0,246769503,1655255265510318082,1655163955956920320,2023-05-07T16:56:23.000Z,replied_to,@DM_holdings Has nothing to do with the econom...,0.0,0.0,3.0,0.0,245.0
0,16435650,1654186791040106498,1654157750560079873,2023-05-04T18:10:38.000Z,replied_to,@imbuyuz The proposed increase looks like one ...,0.0,0.0,1.0,0.0,13.0
0,270834723,1652995999634161666,1652995999634161666,2023-05-01T11:18:52.000Z,None,We celebrate you #TeamPR. Happy Labour Day. ht...,5.0,0.0,20.0,0.0,1109.0
0,58806221,1654728520164057089,1654728520164057089,2023-05-06T06:03:17.000Z,retweeted,RT @SemaUkweliKenya: David Sadera Munyakei: Po...,435.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
0,4895104294,1653637345764159489,1653637345764159489,2023-05-03T05:47:20.000Z,retweeted,RT @UNCCD: Meet our Goodwill Ambassador @baaba...,21.0,0.0,0.0,0.0,0.0
0,17700414,1654435834894905346,1654434054186442752,2023-05-05T10:40:15.000Z,replied_to,@deeRsquared Amepeleka detectives wa Assmio,0.0,0.0,0.0,0.0,27.0
0,1291061156,1655309552311476226,1655309145870876672,2023-05-07T20:32:06.000Z,replied_to,Tiwa Savage stunning at the #CoronationConcert...,0.0,0.0,5.0,0.0,1055.0
0,2497334080,1655266931375501312,1655265090856267779,2023-05-07T17:42:44.000Z,replied_to,@berylsavanah We try again next Sunday,0.0,0.0,1.0,0.0,6.0


In [39]:
df[df['author_id']==1126820633244065792]

,author_id,id,conversation_id,created_at,type,text,retweet_count,reply_count,like_count,quote_count,impression_count


In [ ]:
def africa_report(path_read, datep):
    """Generate report for AfricaCheck"""
    base = path_read.split("/*/")[0]
    df = pd.concat(map(pd.read_excel, glob.glob(f"{path_read}/twitter.xlsx")), axis=1)
    contents = create_list(df, 5, 7, 0)
    n_contents = create_list(df, 6, 7, 0)
    stop = len(df.columns)
    weeks = list(map(str, range(1, int(stop / 7) + 1)))
    n_contents_dict, contents_dict = create_dictionaries(
        contents, n_contents, weeks, "content_w", "n_content_w"
    )
    df = rename_columns(df, contents_dict, n_contents_dict)
    df = drop_duplicated_columns(df)
    df = problematic(df, "treatment", "potentially_problematic", "content_w")
    df.to_excel(f"{base}/{datep}/tracker_twitter.xlsx", index=False)


# def monitor_influencers(days_ago=1):
# --35, 28, 21, 14
def monitor_influencers(days_ago=7, country='KE'):
    """Run process"""
    # Twitter
    _, _, _, bearer_token, _, _,_ = twitter_credentials(
        "../../conf/credentials.yaml"
    )
    participants_tw = get_participants_twitter(country)
    usernames_tw = list(participants_tw["username"])
    path_tw, base, datep = get_paths(days_ago, country)
    create_folder(path_tw)
    scrape_tweets(usernames_tw, f"{path_tw}/", bearer_token, days_ago)
    df_tw = content_twitter(f"{path_tw}")
    create_folder(f"{base}01-preprocessed/content/{datep}")
    df_tw.to_excel(f"{base}01-preprocessed/content/{datep}/tweets.xlsx", index=False)
    count_tw = df_tw.groupby("handle").count()
    count_tw_2 = df_tw.groupby("handle").sum()
    count_tw = count_tw.reset_index()[["handle", "id"]]
    count_tw_2 = count_tw_2.reset_index()[["handle",'public_metrics.impression_count',
                                       'public_metrics.like_count', 'public_metrics.quote_count',
                                       'public_metrics.reply_count', 'public_metrics.retweet_count']]
    count_tw = count_tw.rename({"id": "n_content", "handle": "username",
                                'public_metrics.impression_count': "n_impressions",
                                'public_metrics.like_count':'n_likes', 'public_metrics.quote_count':'n_quotes',
                                'public_metrics.reply_count':'n_replies', 'public_metrics.retweet_count':'n_retweets'}, axis=1)
    count_tw_2 = count_tw_2.rename({"handle": "username",
                                'public_metrics.impression_count': "n_impressions",
                                'public_metrics.like_count':'n_likes', 'public_metrics.quote_count':'n_quotes',
                                'public_metrics.reply_count':'n_replies', 'public_metrics.retweet_count':'n_retweets'}, axis=1)
    count_tw = count_tw.merge(count_tw_2, how="left", on="username")
    found = list(df_tw.handle.unique())
    participants_tw["content"] = np.where(participants_tw["username"].isin(found), 1, 0)
    create_folder(f"{base}01-preprocessed/report/{datep}")
    participants_tw = participants_tw.merge(count_tw, how="left", on="username")
    participants_tw[["n_content","n_impressions", 
                 "n_likes","n_quotes", "n_replies", 
                 "n_retweets"]] = participants_tw[["n_content","n_impressions", 
                                                  "n_likes","n_quotes",
                                                  "n_replies", "n_retweets"]].fillna(0)
    participants_tw.to_excel(
        f"{base}01-preprocessed/report/{datep}/twitter.xlsx", index=False
    )
    summ_tw = participants_tw.groupby(["treatment"]).sum().reset_index()
    summ_tw = summ_tw[["treatment", "content"]]
    summ_tw = summ_tw.rename({"content": "percent_share"}, axis=1)
    summ_tw["percent_share"] = (summ_tw["percent_share"] / 38) * 100
    summ_tw.to_excel(
        f"{base}01-preprocessed/report/{datep}/summary_twitter.xlsx", index=False
    )


if __name__ == "__main__":

    monitor_influencers()
